In [1]:
import itertools
import shutil
from multiprocessing import Pool
from src.recommenders import ImprovedRecommender, PopBasedRecommender
from os import cpu_count
from os.path import exists
from sklearn.decomposition import TruncatedSVD

In [2]:
qual_eval_folder = './evaluation'
items_path = "./data/games_original.pkl"
data_path = "./data/interactions_split_0"

From the base recommender results, we will now continue to use the ideal combination of methods as a baseline before we make further improvements.
This means we use cosine distance and normalized feature vectors. We also don't use feedback weighting.

The following cell runs the basic recommender and provides more in-depth evaluation and compares it to a popularity-based recommender.

In [3]:
rec = PopBasedRecommender(train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet")
rec.generate_recommendations(read_max=1000)
print(rec.evaluate(k=10))
dim_red = TruncatedSVD(n_components=200, random_state=500)
rec = ImprovedRecommender(items_path, train_path=f"{data_path}_train.parquet", test_path=f"{data_path}_test.parquet", val_path=f"{data_path}_val.parquet", sparse=True, tfidf='smooth', use_feedback=False, normalize=True, dim_red=dim_red)
rec.generate_recommendations(read_max=1000)
rec.evaluate(k=10)

{'HR@5': 0.822, 'nDCG@5': 0.3176500479075579, 'recall@5': 0.02807231837180441, 'ideal_recall@5': 0.09193690513421163, 'nRecall@5': 0.31000000000000005}


c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:424: RuntimeWarning: Sparse was set to 'True' but dimensionality reduction is used, using dense matrix representation instead.
  warnings.warn("Sparse was set to 'True' but dimensionality reduction is used, using dense matrix representation instead.", RuntimeWarning)
1000it [00:19, 52.20it/s]


{'HR@5': 0.4,
 'nDCG@5': 0.09676776347960002,
 'recall@5': 0.009264670791775887,
 'ideal_recall@5': 0.09193690513421163,
 'nRecall@5': 0.0988}

In [4]:
import pandas as pd
items = pd.read_pickle(items_path)
items

,publisher,genres,app_name,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore,users_count
0,Valve,[Action],Counter-Strike: Global Offensive,2012-08-21,"[FPS, Multiplayer, Shooter, Action, Team-Based...",NaN,"[Multi-player, Steam Achievements, Full contro...",14.99,False,730,Valve,Very Positive,83,42620
1,Valve,"[Indie, Simulation]",Garry's Mod,2006-11-29,"[Sandbox, Multiplayer, Funny, Moddable, Buildi...",NaN,"[Single-player, Multi-player, Co-op, Cross-Pla...",9.99,False,4000,Facepunch Studios,Overwhelmingly Positive,NA,42157
2,Smartly Dressed Games,"[Action, Adventure, Casual, Free to Play, Indie]",Unturned,2017-07-07,"[Free to Play, Survival, Zombies, Multiplayer,...",NaN,"[Single-player, Online Multi-Player, Online Co...",Free to Play,False,304930,Smartly Dressed Games,Very Positive,NaN,37654
3,Valve,[Action],Left 4 Dead 2,2009-11-16,"[Zombies, Co-op, FPS, Multiplayer, Action, Onl...",NaN,"[Single-player, Multi-player, Co-op, Steam Ach...",19.99,False,550,Valve,Overwhelmingly Positive,89,35990
4,Re-Logic,"[Action, Adventure, Indie, RPG]",Terraria,2011-05-16,"[Sandbox, Adventure, Survival, 2D, Multiplayer...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",9.99,False,105600,Re-Logic,Overwhelmingly Positive,83,28551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7345,Projectile Entertainment,[Indie],Momentum,2016-08-11,"[Indie, Physics]",NaN,"[Single-player, Steam Achievements, Full contr...",9.99,False,462130,Projectile Entertainment,8 user reviews,NaN,1
7346,Nuligine,"[Action, Indie, Racing, Simulation]",MadOut Ice Storm,2015-09-04,"[Racing, Indie, Action, Simulation]",NaN,"[Single-player, Full controller support]",6.99,False,400500,Nuligine,Mixed,NaN,1
7347,Like A Boss LLC,"[Action, Adventure, Indie, Strategy]",OCCHIO,2016-08-11,"[Action, Adventure, Indie, Strategy, Puzzle-Pl...",NaN,"[Single-player, Full controller support]",1.99,False,513140,Acerio Games,7 user reviews,NaN,1
7348,DXF Games,"[Action, Indie, RPG]",Just a Cleric,2016-05-19,"[RPG, Indie, Action, Pixel Graphics, Difficult]",NaN,"[Single-player, Steam Achievements, Steam Trad...",4.99,False,467330,DXF Games,Positive,NaN,1
